In [4]:
!git clone -b atm-10 http://github.com/Zapy67/ATML-PA-4 

fatal: destination path 'ATML-PA-4' already exists and is not an empty directory.


In [40]:
!git pull  http://github.com/Zapy67/ATML-PA-4 atm-10

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 15.25 KiB | 1.39 MiB/s, done.
From http://github.com/Zapy67/ATML-PA-4
 * branch            atm-10     -> FETCH_HEAD
Updating 1e27ef4..36dc983
Fast-forward
 fed_lib/fed_methods.py |   14 +-
 fedsam.ipynb           | 1648 ++++++++++++++++++++++++++++++++++++++++++++----
 2 files changed, 1543 insertions(+), 119 deletions(-)


In [5]:
%cd ATML-PA-4

/kaggle/working/ATML-PA-4


In [6]:
!ls

'ATML - PA 4.pdf'   fed_lib	   LICENSE     task1.ipynb   task3.ipynb
 data		    fedsam.ipynb   README.md   task2.ipynb   task4.ipynb


#  FedSAM on heterogeneous domains

In [46]:
import torch
import torch.nn as nn

In [47]:
import fed_lib
import importlib
importlib.reload(fed_lib)


<module 'fed_lib' from '/kaggle/working/ATML-PA-4/fed_lib/__init__.py'>

In [48]:
!pip install fedlab

In [49]:
from fed_lib import fed_methods, fed_model, utils
importlib.reload(utils)
importlib.reload(fed_methods)
importlib.reload(fed_model)

<module 'fed_lib.fed_model' from '/kaggle/working/ATML-PA-4/fed_lib/fed_model.py'>

In [50]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [51]:
trainset, testset = utils.get_cifar10()

In [52]:
client_loaders = utils.get_heterogenous_domains(trainset, clients=5, balance=False, min_require_size=20, alpha=2, verbose=True)

In [53]:
for idx, loader in enumerate(client_loaders):
    utils.calculate_label_skew(loader.dataset)

Label Distribution (P(Y)): [0.097 0.083 0.095 0.185 0.055 0.115 0.05  0.078 0.136 0.105]
Entropy of Label Distribution H(P(Y)): 2.1396122755658813
Normalized Entropy of Label Distribution: 0.929221804690722

Label Distribution (P(Y)): [0.139 0.208 0.099 0.044 0.04  0.172 0.064 0.078 0.155]
Entropy of Label Distribution H(P(Y)): 1.9788088645208894
Normalized Entropy of Label Distribution: 0.9005947252419123

Label Distribution (P(Y)): [0.031 0.024 0.041 0.169 0.049 0.084 0.23  0.085 0.093 0.194]
Entropy of Label Distribution H(P(Y)): 1.9781868035447743
Normalized Entropy of Label Distribution: 0.8591156129533275

Label Distribution (P(Y)): [0.18  0.026 0.148 0.025 0.111 0.066 0.094 0.215 0.024 0.112]
Entropy of Label Distribution H(P(Y)): 1.9945229649691025
Normalized Entropy of Label Distribution: 0.866210317715394

Label Distribution (P(Y)): [0.035 0.155 0.122 0.083 0.29  0.043 0.055 0.033 0.082 0.101]
Entropy of Label Distribution H(P(Y)): 1.9798053214127318
Normalized Entropy of Lab

In [54]:
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]
domains = None

In [55]:
client_weights

[0.19852, 0.22988, 0.20568, 0.20318, 0.16274]

In [56]:
partition = "dirichlet"
alpha = 0.1
num_clients = 5
domains=None
fed = fed_model.Federation(num_clients, None, partition, domains, alpha, device, batch_size=64, pin_memory=True, num_workers=2)

In [57]:
client_loaders = fed.client_dataloaders
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]

In [ ]:
fed.set_method(fed_methods.FedSAM(sam_rho=0, 
             num_local_steps=len(trainset)/num_clients, 
             client_aggregation_weights= client_weights))

In [59]:
fed.train(25, lr=1e-3, verbose=False)


--- Round 1/25 ---
Training Clients
Training Client 1/5
Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 1:


FedSAM  | Test Loss: nan, Test Acc: 10.00%

--- Round 2/25 ---
Training Clients
Training Client 1/5


Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 2:


FedSAM  | Test Loss: nan, Test Acc: 10.00%

--- Round 3/25 ---
Training Clients
Training Client 1/5


Exception in thread Thread-226 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/pin_memory.py", line 59, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/pin_memory.py", line 35, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/multiprocessing/reductions.py", line 541, in rebuild_storage_fd
    fd = df.detach()
         ^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/resource_

KeyboardInterrupt: 

In [127]:
fed.clients[0]

SmallCNN(
  (backbone): Sequential(
    (0): SmallConvBlock(in_channels=3, out_channels=32)
    (1): SmallConvBlock(in_channels=32, out_channels=64)
    (2): AvgPool2d(kernel_size=1, stride=1, padding=0)
  )
  (task_head): Sequential(
    (0): Linear(in_features=64, out_features=10, bias=True)
  )
)